# Import required packages

In [ ]:
import Functions

import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from matplotlib.colors import ListedColormap, BoundaryNorm
from matplotlib.animation import FuncAnimation
import matplotlib.animation as animation

# Level 1 Simulation

In [ ]:
def Level_1_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant):
    # initialize price list for market
    price_list = [initial_price]
    # initialize transaction quantities. Note that this implies at t=0 we initialize with imitators not trading and fundamentalists changing to the updated price values
    transaction_quantities = Functions.next_state(trader_grid, np.zeros((L,L)), price_list[-1], fundamental_value, L)
    transactions = [transaction_quantities]
    # update transaction quantities for time steps
    for t in range(time):
        transactions.append(Functions.next_state(trader_grid, transactions[-1], price_list[-1], fundamental_value, L))
        trans_quantity = Functions.calculation_transaction_quantity(transactions[-1], L)
        price_list.append(Functions.price_function(price_list[-1],sensitivity_constant,L, trans_quantity))
    # return array of transaction quantities and prices over time
    return np.array(transactions), np.array(price_list)


### Set initial conditions for Level 1

In [ ]:
fundamentalists_probabilities = [0.2, 0.8]
L = 50
fundamental_value = 100
initial_price = 105
time = 100
constant = 0.5

### Results

In [ ]:
# plot prices for different distributions of fundamentalist and imitator
plt.figure(dpi = 300)
for p in fundamentalists_probabilities:
    trader_grid = Functions.grid_stock_market(L, p)
    prices = Level_1_simulation(trader_grid, initial_price, fundamental_value, time, L, constant)
    plt.plot(np.arange(time + 1), prices[1], label = p)
plt.suptitle('Stock prices for varying distributions of fundamentalists and imitators')
plt.xlabel('Time')
plt.ylabel('Stock price')
plt.legend()
plt.show()
plt.close()

In [ ]:
# Generate trader grid for presentation
trader_grid_presentation = Functions.grid_stock_market(L, 0.5)
# Plot trader type distribution
plt.figure(dpi = 300)
im = plt.imshow(trader_grid_presentation, origin="lower")
ax = plt.gca();
# Minor ticks
ax.set_xticks(np.arange(-.5, 50, 1), minor=True)
ax.set_yticks(np.arange(-.5, 50, 1), minor=True)
# Gridlines based on minor ticks
ax.grid(which='minor', color='black', linestyle='-', linewidth=0.2)
#plt.title('Representation of the trader grid')
plt.show()

# Level 2 Simulation

In [ ]:
def Level_2_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, news_relevance):
    # initialize price list for market
    price_list = [initial_price]
    
    # initialize transaction quantities. Note that this implies at t=0 we initialize with imitators not trading and fundamentalists changing to the updated price values
    transaction_quantities = Functions.next_state_Level_2(trader_grid, np.zeros((L,L)), price_list, fundamental_value, news_relevance, L)
    transactions = [transaction_quantities]
    # iterate through time steps and calculate next stage of CA iteration using the previous functions
    for t in range(time):
        transactions.append(Functions.next_state_Level_2(trader_grid, transactions[-1], price_list, fundamental_value, news_relevance, L))
        trans_quantity = Functions.calculation_transaction_quantity(transactions[-1], L)
        price_list.append(Functions.price_function(price_list[-1],sensitivity_constant,L, trans_quantity))
    # return array of transaction quantities per trader at each time step and the prices at each time step 
    return np.array(transactions), np.array(price_list) 


### Set initial conditions for Level 2

In [ ]:
L = 10
fundamental_value = 100
initial_price = 100
time = 100
constant = 0.05
trader_grid = Functions.grid_stock_market(L, 0.3)
news_relevance = [0.2, 0.7]

### Results

In [ ]:
resultS = Level_2_simulation(trader_grid, initial_price, fundamental_value, time, L, constant, news_relevance)
plt.figure(dpi = 300)
plt.plot(np.arange(time+1), resultS[1])
plt.show()
plt.close()

# Level 3 Simulation

In [ ]:
def Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, constant_trading, news_relevance, stock_favorability, period_length):
    # initialize price list, price fluctuation storage, and trading activity for market
    price_list = [initial_price]
    price_fluctuation = Functions.price_fluctuations(period_length, price_list)
    price_fluctuation_list = [price_fluctuation]
    trades = Functions.trading_activity_function(constant_trading, price_fluctuation, stock_favorability)
    trades_list = [trades]
    
    # initialize transaction quantities. Note that this implies at t=0 we initialize with imitators not trading and fundamentalists changing to the updated price values
    transaction_quantities = Functions.next_state_Level_3(trader_grid, np.zeros((L,L)), price_list, fundamental_value, news_relevance, L, trades)
    transactions = [transaction_quantities[0]]
    News = [transaction_quantities[1]]
    # iterate CA model through time using next_state_level_3 function
    for t in range(time):
        # calulcate price fluctuation and trading activity at this time step
        price_fluctuation = Functions.price_fluctuations(period_length, price_list)
        price_fluctuation_list.append(price_fluctuation)
        trades = Functions.trading_activity_function(constant_trading, price_fluctuation, stock_favorability)
        trades_list.append(trades)
        
        # continue to next step in cellular automata and append next results
        next_result = Functions.next_state_Level_3(trader_grid, transactions[-1], price_list, fundamental_value, news_relevance, L, trades)
        transactions.append(next_result[0])
        News.append(next_result[1])
        trans_quantity = Functions.calculation_transaction_quantity(transactions[-1], L)
        price_list.append(Functions.price_function(price_list[-1],sensitivity_constant,L, trans_quantity))
    # return results as arrays: transaction matrix, prices, fluctuations in price, trading activity, and news relevance for fundamentalists and imitators
    return np.array(transactions), np.array(price_list), np.array(price_fluctuation_list), np.array(trades_list), np.array(News)

### Set initial conditions for Level 3

In [ ]:
L = 50
fundamental_value = 100
initial_price = 100
time = 500
sensitivity_constant = 0.7
trader_grid = Functions.grid_stock_market(L, 0.5)
trading_constant = 20
news_relevance = [0.2, 0.7]
stock = 0.01 
period = 10

### Results 

In [ ]:
resultS = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
plt.figure(dpi = 300)
plt.plot(np.arange(time+1), resultS[1], color = 'purple')
plt.xlabel('Time')
plt.ylabel('Price')
plt.suptitle('Price fluctuations of a single Stock')
plt.show()
plt.close()

plt.figure(dpi = 300)
plt.plot(np.arange(time+1), resultS[3], color = 'red')
plt.xlabel('Time')
plt.ylabel('Trading activity')
plt.suptitle('Trading activity')
plt.show()
plt.close()

In [ ]:
# iterate through news relevance for fundamentalists
Fundamentalist_news = np.arange(0,1.1, 0.1) 

# Generate slider plot for various news relevances  
fig = make_subplots(rows=2, cols=1,shared_xaxes=True)
# Add traces, one for each slider step
for news_fundamentalists in Fundamentalist_news:
    news_relevance = [news_fundamentalists, 0.5]
    results = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="purple", width=2),
            name="News" + str(news_fundamentalists),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[1]
            ),row=1, col=1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="red", width=2),
            name="News " + str(news_fundamentalists),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[3]
            ),row=2, col=1)


# Create and add slider
steps = []

for i in range(0, len(fig.data), 2):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i:i+2] = [True, True]
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time:  "},
    pad={"t": 50},
    steps=steps
)]

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Trading Activity", row=2, col=1)
fig.update_layout(sliders=sliders, title="Prices & Trading Activity for varying News relevance", template ="plotly_white")

plotly.offline.plot(fig, filename='Phases_NewsRelevance.html')
fig.show() 

In [ ]:
# Initialize storage of order parameters
average_price = []
price_variance = []
average_trading_activity = []
trading_activity_variance = []

# Iterate through different sensitivity constants
news_relevance = [0.2, 0.7]
sensitivity_variations = np.arange(0.2, 1.4, 0.2)
# Generate order parameters for varying control parameter
for sensitivity in sensitivity_variations:
    price_storage = []
    trading_activity_storage = []
    for statistical_significance in range(30):
        simulation_result = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity, trading_constant, news_relevance, stock, period)
        price_storage.append(np.mean(simulation_result[1]))
        trading_activity_storage.append(np.mean(simulation_result[3]))
    average_price.append(np.mean(price_storage))
    price_variance.append(np.std(price_storage))
    average_trading_activity.append(np.mean(trading_activity_storage))
    trading_activity_variance.append(np.std(trading_activity_storage))

# calculate confidence intervals 
ci_price = 1.96 * np.array(price_variance)/30
ci_trading_activity = 1.96 * np.array(trading_activity_variance)/30

# plot results 
plt.figure(dpi = 300, figsize = (4, 8))
plt.subplot(211)
plt.plot(sensitivity_variations, average_price, color = 'purple')
plt.fill_between(sensitivity_variations, (np.array(average_price)-ci_price), (np.array(average_price)+ci_price), color='purple', alpha=.25)
plt.ylabel('Mean price')
plt.subplot(212)
plt.plot(sensitivity_variations, average_trading_activity, color = 'red')
plt.fill_between(sensitivity_variations, (np.array(average_trading_activity)-ci_trading_activity), (np.array(average_trading_activity)+ci_trading_activity), color='red', alpha=.25)
plt.ylabel('Mean trading activity')
plt.xlabel('Sensitivity constant')
plt.suptitle('Emergence of Price fluctuations and Trading activity')
plt.show()
plt.close()

In [ ]:
# re-initialize parameter
sensitivity_constant = 0.7

# re-initialize storage
average_price = []
price_variance = []
average_trading_activity = []
trading_activity_variance = []

# Generate order parameters for varying control parameter
Fundamentalist_news = np.arange(0,1.1, 0.1) 
for fundamentalist_news in Fundamentalist_news:
    news_relevance = [fundamentalist_news, 0.5]
    price_storage = []
    trading_activity_storage = []
    for statistical_significance in range(30):
        simulation_result = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity, trading_constant, news_relevance, stock, period)
        price_storage.append(np.mean(simulation_result[1]))
        trading_activity_storage.append(np.mean(simulation_result[3]))
    average_price.append(np.mean(price_storage))
    price_variance.append(np.std(price_storage))
    average_trading_activity.append(np.mean(trading_activity_storage))
    trading_activity_variance.append(np.std(trading_activity_storage))

# calculate confidence intervals 
ci_price = 1.96 * np.array(price_variance)/30
ci_trading_activity = 1.96 * np.array(trading_activity_variance)/30
# generate plots 
plt.figure(dpi = 300, figsize = (4, 8))
plt.subplot(211)
plt.plot(Fundamentalist_news, average_price, color = 'purple')
plt.fill_between(Fundamentalist_news, (np.array(average_price)-ci_price), (np.array(average_price)+ci_price), color='purple', alpha=.25)
plt.ylabel('Mean price')
plt.subplot(212)
plt.plot(Fundamentalist_news, average_trading_activity, color = 'red')
plt.fill_between(Fundamentalist_news, (np.array(average_trading_activity)-ci_trading_activity), (np.array(average_trading_activity)+ci_trading_activity), color='red', alpha=.25)
plt.ylabel('Mean trading activity')
plt.xlabel('Factor of News relevance for Fundamentalists')
plt.suptitle('Emergence of Price fluctuation and Trading activity')
plt.show()
plt.close()

In [ ]:
### Map of different regimes for News relevance for fundamentalitsts and sensitivity constant  
# initialize storage for parameters, mean prices, and trading activity
Prices_matrix = []
Trading_activity_matrix = []

# initialize fundamentalist probability variations & iterate through different probabilities
Fundamentalist_news = np.arange(0,1.1, 0.1)
for fundamentalist_probability in Fundamentalist_news:
    news_relevance = [fundamentalist_probability, 0.5]
    # variate sensitivity constant
    Prices = []
    Trading = []
    for sensitivity_constant in sensitivity_variations:
        # initialize storage for each set of parameters and simulation results
        price_storage = []
        trading_activity_storage = []
        # simulate 30 times per parameter set for statistical significance
        for statistical_significance in range(3):
            simulation_results = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
            price_storage.append(np.mean(simulation_results[1]))
            trading_activity_storage.append(np.mean(simulation_results[3]))
        # append all results and parameters
        Prices.append(np.mean(price_storage))
        Trading.append(np.mean(trading_activity_storage))
    Prices_matrix.append(np.array(Prices))
    Trading_activity_matrix.append(np.array(Trading))


transitions_price = np.array(Prices_matrix)
transitions_trading_activity = np.array(Trading_activity_matrix)


# Generate phase transition map depending on sensitivity constant and fraction of fundamentalists
plt.imshow(transitions_trading_activity, extent=[min(sensitivity_variations), max(sensitivity_variations), min(Fundamentalist_news), max(Fundamentalist_news)], origin='lower', cmap='plasma_r')
plt.colorbar(label='Mean Trading Activity')
# Add labels and title
plt.xlabel('Sensitivity Constant')
plt.ylabel('News relevance for Fundamentalists')
plt.title('Phase Transition Map')
# Show the plot
plt.show()

In [ ]:
# Map of different regimes for relevance of news for fundamentalitsts and imitators
sensitivity_constant = 0.7

# initialize storage for parameters, mean prices, and trading activity
Prices_matrix = []
Trading_activity_matrix = []

# initialize fundamentalist probability variations & iterate through different probabilities
Imitator_news = np.arange(0,1.3, 0.1)
for fundamentalist_probability in Fundamentalist_news:
    # variate sensitivity constant
    Prices = []
    Trading = []
    for imitators_probability in Imitator_news:
        news_relevance = [fundamentalist_probability, imitators_probability]
        # initialize storage for each set of parameters and simulation results
        price_storage = []
        trading_activity_storage = []
        # simulate 30 times per parameter set for statistical significance
        for statistical_significance in range(3):
            simulation_results = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
            price_storage.append(np.mean(simulation_results[1]))
            trading_activity_storage.append(np.mean(simulation_results[3]))
        # append all results and parameters
        Prices.append(np.mean(price_storage))
        Trading.append(np.mean(trading_activity_storage))
    Prices_matrix.append(np.array(Prices))
    Trading_activity_matrix.append(np.array(Trading))


transitions_price = np.array(Prices_matrix)
transitions_trading_activity = np.array(Trading_activity_matrix)


# Generate phase transition map depending on sensitivity constant and fraction of fundamentalists
plt.imshow(transitions_trading_activity, extent=[min(Imitator_news), max(Imitator_news), min(Fundamentalist_news), max(Fundamentalist_news)], origin='lower', cmap='plasma_r')
plt.colorbar(label='Mean Trading Activity')
# Add labels and title
plt.xlabel('News relevance for Imitators')
plt.ylabel('News relevance for Fundamentalists')
plt.title('Phase Transition Map')
# Show the plot
plt.show()

In [ ]:
# Initialize model parameters
L = 50
fundamental_value = 100
initial_price = 100
time = 500
sensitivity_constant = 0.7
trading_constant = 20
news_relevance = [0.2, 0.7]
stock = 0.01 
period = 10

# initialize storage for parameters, mean prices, and trading activity
fundamentalist_probabilities = [i / 100 for i in range(10, 101, 5)]
average_price = []
price_stds = []
average_trading_activity = []
trading_activity_stds = []

for i in fundamentalist_probabilities:
    trader_grid = Functions.grid_stock_market(L, i)
    price_storage = []
    trading_activity_storage = []
    for j in range(30):
        resultS = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
        price_storage.append(np.mean(resultS[1]))
        trading_activity_storage.append(np.mean(resultS[3]))
    average_price.append(np.mean(price_storage))
    price_stds.append(np.std(price_storage))
    average_trading_activity.append(np.mean(trading_activity_storage))
    trading_activity_stds.append(np.std(trading_activity_storage))
    
ci_price = 1.96 * np.array(price_stds)/30
ci_trading_activity = 1.96 * np.array(trading_activity_stds)/30

plt.figure(figsize = (4, 5))

plt.plot(fundamentalist_probabilities, average_trading_activity, color = 'red')
plt.fill_between(fundamentalist_probabilities, (np.array(average_trading_activity)-ci_trading_activity), (np.array(average_trading_activity)+ci_trading_activity), color='red', alpha=.25)
plt.ylabel('Mean trading activity')
plt.xlabel('Fundamentalist probability')
plt.show()
plt.close()

In [ ]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True) 

# Add traces, one for each slider step
for i in fundamentalist_probabilities:
    trader_grid = Functions.grid_stock_market(L, i)
    results = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
    fig.add_trace(
        go.Scatter(
            visible = False,
            line = dict(color="purple", width=2),
            name = "Fundamentalist prob." + str(i),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[1]
            ),row=1, col=1)
    fig.add_trace(
        go.Scatter(
            visible = False,
            line = dict(color="red", width=2),
            name = "Fundamentalist prob. " + str(i),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[3]
            ),row=2, col=1)


# Create and add slider
steps = []

for i in range(0, len(fig.data), 2):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i:i+2] = [True, True]
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time:  "},
    pad={"t": 50},
    steps=steps
)]

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Trading Activity", row=2, col=1)
fig.update_layout(sliders=sliders, title="Prices & Trading Activity for varying sensitivity constant", template ="plotly_white")
plotly.offline.plot(fig, filename='Phases_SensitivityConstant.html')
fig.show() 

### Stylized facts

#### Autocorrelation

In [ ]:
# Initialize model parameters
L = 50
fundamental_value = 100
initial_price = 100
time = 500
sensitivity_constant = 0.7
trader_grid = Functions.grid_stock_market(L, 0.5)
trading_constant = 20
news_relevance = [0.2, 0.7]
stock = 0.01 
period = 10


resultS = Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
returns = Functions.calculate_returns(resultS[1])
autocorr = np.correlate(returns, returns, mode='full')
selected_lags = range(1, 101)
autocorr = autocorr[len(returns)-1:len(returns)-1-len(selected_lags):-1]

#### Normalized returns

In [ ]:
sns.histplot(returns, kde=True, bins=30)
plt.title('Distribution of Returns')
plt.xlabel('Returns')
plt.ylabel('Frequency')
plt.show()

#### Volatility Clustering

#### YUAN PART GOES HERE

# Model Twist Simulation

In [ ]:
def random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, constant_trading, news_relevance, stock_favorability, period_length):
    # initialize price list for market
    price_list = [initial_price]
    price_fluctuation = Functions.price_fluctuations(period_length, price_list)
    price_fluctuation_list = [price_fluctuation]
    trades = Functions.trading_activity_function(constant_trading, price_fluctuation, stock_favorability)
    trades_list = [trades]
    
    # Initialize neighbour they are looking at
    random_array_row = [np.random.randint(0, L-1) for _ in range(L)]
    random_array_column = [np.random.randint(0, L-1) for _ in range(L)]
    
    # initialize transaction quantities. Note that this implies at t=0 we initialize with imitators not trading and fundamentalists changing to the updated price values
    transaction_quantities = Functions.next_state_random_neighbourhood(trader_grid, np.zeros((L,L)), price_list, fundamental_value, news_relevance, L, trades, random_array_row, random_array_column)
    transactions = [transaction_quantities]
    
    for t in range(time):
        price_fluctuation = Functions.price_fluctuations(period_length, price_list)
        price_fluctuation_list.append(price_fluctuation)
        trades = Functions.trading_activity_function(constant_trading, price_fluctuation, stock_favorability)
        trades_list.append(trades)
        
        transactions.append(Functions.next_state_random_neighbourhood(trader_grid, transactions[-1], price_list, fundamental_value, news_relevance, L, trades, random_array_row, random_array_column))
        trans_quantity = Functions.calculation_transaction_quantity(transactions[-1], L)
        price_list.append(Functions.price_function(price_list[-1],sensitivity_constant,L, trans_quantity))
    return np.array(transactions), np.array(price_list), np.array(price_fluctuation_list), np.array(trades_list)


### Set initial conditions for the Model Twist

In [ ]:
# same as for Level 3 simulation
L = 50
fundamental_value = 100
initial_price = 100
time = 500
sensitivity_constant = 0.7
trader_grid = Functions.grid_stock_market(L, 0.5)
trading_constant = 20
news_relevance = [0.2, 0.5]
stock = 0.01 
period = 10

### Results

In [ ]:
resultS = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
plt.figure(dpi = 300)
plt.plot(np.arange(time+1), resultS[1], color = 'purple')
plt.xlabel('Time')
plt.ylabel('Price')
plt.suptitle('Price fluctuations of a single Stock')
plt.show()
plt.close()

plt.figure(dpi = 300)
plt.plot(np.arange(time+1), resultS[3], color = 'red')
plt.xlabel('Time')
plt.ylabel('Trading activity')
plt.suptitle('Trading activity')
plt.show()
plt.close()

In [ ]:
Prices_matrix = []
Trading_activity_matrix = []

# initialize fundamentalist probability variations & iterate through different probabilities
Fundamentalist_news = np.arange(0,1.1, 0.1)
sensitivity_variations = np.arange(0, 1.1, 0.1)
for fundamentalist_probability in Fundamentalist_news:
    news_relevance = [fundamentalist_probability, 0.5]
    # variate sensitivity constant
    Prices = []
    Trading = []
    for sensitivity_constant in sensitivity_variations:
        # initialize storage for each set of parameters and simulation results
        price_storage = []
        trading_activity_storage = []
        # simulate 30 times per parameter set for statistical significance
        for statistical_significance in range(5):
            simulation_results = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
            price_storage.append(np.mean(simulation_results[1]))
            trading_activity_storage.append(np.mean(simulation_results[3]))
        # append all results and parameters
        Prices.append(np.mean(price_storage))
        Trading.append(np.mean(trading_activity_storage))
    Prices_matrix.append(np.array(Prices))
    Trading_activity_matrix.append(np.array(Trading))

transitions_price = np.array(Prices_matrix)
transitions_trading_activity = np.array(Trading_activity_matrix)

plt.figure(dpi = 300)
# Generate phase transition map depending on sensitivity constant and fraction of fundamentalists
plt.imshow(transitions_trading_activity, extent=[min(sensitivity_variations), max(sensitivity_variations), min(Fundamentalist_news), max(Fundamentalist_news)], origin='lower', cmap='plasma_r')
plt.colorbar(label='Mean Trading Activity')
# Add labels and title
plt.xlabel('Sensitivity Constant')
plt.ylabel('News relevance for Fundamentalists')
plt.title('Phase Transition Map')
# Show the plot
plt.show()

In [ ]:
news_relevance = [0.2, 0.5]

average_price = []
price_variance = []
average_trading_activity = []
trading_activity_variance = []

# Generate order parameters for varying control parameter
for sensitivity in sensitivity_variations:
    price_storage = []
    trading_activity_storage = []
    for statistical_significance in range(30):
        simulation_result = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity, trading_constant, news_relevance, stock, period)
        price_storage.append(np.mean(simulation_result[1]))
        trading_activity_storage.append(np.mean(simulation_result[3]))
    average_price.append(np.mean(price_storage))
    price_variance.append(np.std(price_storage))
    average_trading_activity.append(np.mean(trading_activity_storage))
    trading_activity_variance.append(np.std(trading_activity_storage))

ci_price = 1.96 * np.array(price_variance)/30
ci_trading_activity = 1.96 * np.array(trading_activity_variance)/30

plt.figure(dpi = 300, figsize = (4, 8))
plt.subplot(211)
plt.plot(sensitivity_variations, average_price, color = 'purple')
plt.fill_between(sensitivity_variations, (np.array(average_price)-ci_price), (np.array(average_price)+ci_price), color='purple', alpha=.25)
plt.ylabel('Mean price')
plt.subplot(212)
plt.plot(sensitivity_variations, average_trading_activity, color = 'red')
plt.fill_between(sensitivity_variations, (np.array(average_trading_activity)-ci_trading_activity), (np.array(average_trading_activity)+ci_trading_activity), color='red', alpha=.25)
plt.ylabel('Mean trading activity')
plt.xlabel('Sensitivity constant')
plt.show()
plt.close()

In [ ]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True) # ,vertical_spacing  = 0.25
# Add traces, one for each slider step
for sensitivity in sensitivity_variations:
    results = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity, trading_constant, news_relevance, stock, period)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="purple", width=2),
            name="Sensitivity" + str(sensitivity),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[1]
            ),row=1, col=1)
    fig.add_trace(
        go.Scatter(
            visible=False,
            line=dict(color="red", width=2),
            name="Sensitivity " + str(sensitivity),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[3]
            ),row=2, col=1)
    

# Create and add slider
steps = []

for i in range(0, len(fig.data), 2):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i:i+2] = [True, True]
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time:  "},
    pad={"t": 50},
    steps=steps
)]

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Trading Activity", row=2, col=1)
fig.update_layout(sliders=sliders, title="Prices & Trading Activity for varying sensitivity constant", template ="plotly_white")

plotly.offline.plot(fig, filename='Phases_SensitivityConstant.html')
fig.show() 

In [ ]:
# Initialize model parameters
L = 50
fundamental_value = 100
initial_price = 100
time = 500
sensitivity_constant = 0.7
trading_constant = 20
news_relevance = [0.2, 0.7]
stock = 0.01 
period = 10

# initialize storage for parameters, mean prices, and trading activity
fundamentalist_probabilities = [i / 100 for i in range(10, 101, 5)]
average_price = []
price_stds = []
average_trading_activity = []
trading_activity_stds = []

for i in fundamentalist_probabilities:
    trader_grid = Functions.grid_stock_market(L, i)
    price_storage = []
    trading_activity_storage = []
    for j in range(30):
        resultS = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
        price_storage.append(np.mean(resultS[1]))
        trading_activity_storage.append(np.mean(resultS[3]))
    average_price.append(np.mean(price_storage))
    price_stds.append(np.std(price_storage))
    average_trading_activity.append(np.mean(trading_activity_storage))
    trading_activity_stds.append(np.std(trading_activity_storage))
    
ci_price = 1.96 * np.array(price_stds)/30
ci_trading_activity = 1.96 * np.array(trading_activity_stds)/30

plt.figure(figsize = (4, 5))

plt.plot(fundamentalist_probabilities, average_trading_activity, color = 'red')
plt.fill_between(fundamentalist_probabilities, (np.array(average_trading_activity)-ci_trading_activity), (np.array(average_trading_activity)+ci_trading_activity), color='red', alpha=.25)
plt.ylabel('Mean trading activity')
plt.xlabel('Fundamentalist probability')
plt.show()
plt.close()

In [ ]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True) 

# Add traces, one for each slider step
for i in fundamentalist_probabilities:
    trader_grid = Functions.grid_stock_market(L, i)
    resultS = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
    fig.add_trace(
        go.Scatter(
            visible = False,
            line = dict(color="purple", width=2),
            name = "Fundamentalist prob." + str(i),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[1]
            ),row=1, col=1)
    fig.add_trace(
        go.Scatter(
            visible = False,
            line = dict(color="red", width=2),
            name = "Fundamentalist prob. " + str(i),
            x = np.arange(0, len(results[1])+1, 1.0),
            y = results[3]
            ),row=2, col=1)


# Create and add slider
steps = []

for i in range(0, len(fig.data), 2):
    step = dict(
        method="restyle",
        args=["visible", [False] * len(fig.data)],
    )
    step["args"][1][i:i+2] = [True, True]
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time:  "},
    pad={"t": 50},
    steps=steps
)]

fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="Trading Activity", row=2, col=1)
fig.update_layout(sliders=sliders, title="Prices & Trading Activity for varying sensitivity constant", template ="plotly_white")
plotly.offline.plot(fig, filename='Phases_SensitivityConstant.html')
fig.show() 

### Stylized facts

#### Autocorrelation

In [ ]:
# Initialize model parameters
L = 50
fundamental_value = 100
initial_price = 100
time = 500
sensitivity_constant = 0.7
trader_grid = Functions.grid_stock_market(L, 0.5)
trading_constant = 20
news_relevance = [0.2, 0.7]
stock = 0.01 
period = 10


resultS = random_neighbourhood_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, trading_constant, news_relevance, stock, period)
returns = Functions.calculate_returns(resultS[1])
autocorr = np.correlate(returns, returns, mode='full')
selected_lags = range(1, 101)
autocorr = autocorr[len(returns)-1:len(returns)-1-len(selected_lags):-1]

#### Normalized returns

In [ ]:
sns.histplot(returns, kde=True, bins=30)
plt.title('Distribution of Returns')
plt.xlabel('Returns')
plt.ylabel('Frequency')
plt.show()

#### Volatility Clustering

#### YUAN PART GOES HERE

# Plot for Baseline Model: Fundamentalist Ratio = 0.2 & 0.8

In [ ]:
def Level_1_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_contant):
    """
        Parameters:
        - trader_grid (np.ndarray): A grid representing the distribution of traders in the market. Each cell contains a trader type.
        - initial_price (float): The initial price of the stock at the beginning of the simulation.
        - fundamental_value (float): The perceived fundamental value of the stock, which influences the trading decisions of fundamentalists.
        - time (int): The number of time steps for which the simulation is run.
        - L (int): The size of one dimension of the square trader grid. The total number of traders is L x L.
        - sensitivity_constant (float): A constant that determines the sensitivity of price changes to transaction volumes.

        Outputs:
        - transactions (np.ndarray): An array of transaction quantities for each time step, indicating the market activity.
        - price_list (np.ndarray): An array of stock prices at each time step throughout the simulation.

    """
    # initialize price list for market
    price_list = [initial_price]
    # initialize transaction quantities. 
    # Note that this implies at t=0 we initialize with imitators not trading and fundamentalists changing to the updated price values
    transaction_quantities = Functions.next_state(trader_grid, np.zeros((L,L)), price_list[-1], fundamental_value, L)
    transactions = [transaction_quantities]
    for t in range(time):
        transactions.append(Functions.next_state(trader_grid, transactions[-1], price_list[-1], fundamental_value, L))
        trans_quantity = Functions.calculation_transaction_quantity(transactions[-1], L)
        price_list.append(Functions.price_function(price_list[-1],sensitivity_contant,L, trans_quantity))
    return np.array(transactions), np.array(price_list)

# Parameters 
fundamentalists_probabilities = [0.2, 0.8]
L = 100
fundamental_value = 100
initial_price = 105
time = 100
constant = 0.5

# Verification plots for Baseline model
plt.figure(dpi = 300)
for p in fundamentalists_probabilities:
    trader_grid = Functions.grid_stock_market(L, p)
    prices = Level_1_simulation(trader_grid, initial_price, fundamental_value, time, L, constant)
    plt.plot(np.arange(time + 1), prices[1], label = p)
plt.suptitle('Stock prices for varying distributions of fundamentalists and imitators')
plt.xlabel('Time')
plt.ylabel('Stock price')
plt.legend()
plt.show()
plt.close()
    

trader_grid_presentation = Functions.grid_stock_market(L, 0.2)
# Plot trader type distribution
im = plt.imshow(trader_grid_presentation)
plt.show()

# Baseline CA Animation
- Saves the output as an mp4 file.

In [ ]:
# Animation function
def animate(i):
    """
        Parameters:
        - i (int): The current frame index (time step in the simulation).

        Returns:
        - list: A list containing the updated cellular automaton image and price line objects.
    """
    # Update the CA image
    ca_im.set_array(Level_1_simulation(trader_grid, initial_price, fundamental_value, i+1, L, constant)[0][-1])
    # Update the price line
    price_line.set_data(np.arange(i+1), Level_1_simulation(trader_grid, initial_price, fundamental_value, i+1, L, constant)[1][:i+1])
    return [ca_im, price_line]

# Set the parameters for the simulation
L = 10
fundamental_value = 100
initial_price = 105
time = 100
constant = 0.8
trader_grid = Functions.grid_stock_market(L, 0.2)

# Set the minimum and maximum price limits for the plot
price_min = initial_price - 10
price_max = initial_price + 5

# Create a figure for animation
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))
ca_axes = axes[0]
ca_axes.set_title('Cellular Automaton (Level 1)')
ca_im = ca_axes.imshow(trader_grid, animated=True, cmap='plasma_r', interpolation='none', aspect='auto')

# Price subplot
price_axes = axes[1]
price_axes.set_title('Price Over Time')
price_axes.set_xlabel('Time')
price_axes.set_ylabel('Price')
price_line, = price_axes.plot([], [], lw=2)
price_axes.set_xlim(0, time)
price_axes.set_ylim(price_min, price_max)

# Create the animation
ani = FuncAnimation(fig, animate, frames=time, interval=250, blit=True)

# Show the animation
plt.tight_layout()
plt.show()

# Save the animation
ani.save('cellular_automaton_level_1.mp4', writer='ffmpeg')

# Exapnded Model

In [ ]:
def Level_3_simulation(trader_grid, initial_price, fundamental_value, time, L, sensitivity_constant, constant_trading, news_relevance, stock_favorability, period_length):
    """
        Parameters:
        Uses the same parameters as stated in level 1 plus some extra
        - constant_trading (float): A constant representing the base level of trading activity in the market.
        - news_relevance (float): A measure of how relevant news is to the market, influencing traders' decisions.
        - stock_favorability (float): A measure of the overall favorability of the stock among traders.
        - period_length (int): The length of the period over which price fluctuations are considered.


        Returns:
        - tuple: A tuple containing four lists:
            - transactions (list of numpy.ndarray): A list of arrays representing the transaction quantities at each time step.
            - price_list (list of float): A list of stock prices at each time step.
            - price_fluctuation_list (list of float): A list of price fluctuations at each period.
            - trades_list (list of float): A list of trading activity levels at each time step.

    """
    
    price_list = [initial_price]
    price_fluctuation = Functions.price_fluctuations(period_length, price_list)
    price_fluctuation_list = [price_fluctuation]
    trades = Functions.trading_activity_function(constant_trading, price_fluctuation, stock_favorability)
    trades_list = [trades]
    
    transaction_quantities, news_both = Functions.next_state_Level_3(trader_grid, np.zeros((L, L)), price_list, fundamental_value, news_relevance, L, trades)
    transactions = [transaction_quantities]
    
    for t in range(time):
        price_fluctuation = Functions.price_fluctuations(period_length, price_list)
        price_fluctuation_list.append(price_fluctuation)
        trades = Functions.trading_activity_function(constant_trading, price_fluctuation, stock_favorability)
        trades_list.append(trades)
        
        transaction_quantities, _ = Functions.next_state_Level_3(trader_grid, transactions[-1], price_list, fundamental_value, news_relevance, L, trades)
        transactions.append(transaction_quantities)
        
        trans_quantity = Functions.calculation_transaction_quantity(transaction_quantities, L)
        price_list.append(Functions.price_function(price_list[-1], sensitivity_constant, L, trans_quantity))
    
    return transactions, price_list, price_fluctuation_list, trades_list


def process_transactions(transactions, trader_grid, price_list, fundamental_value, L):
    """
        Parameters:
        - transactions (numpy.ndarray): A 3D array representing the transaction states at each 
          timestep for each cell in the grid.
        - trader_grid (numpy.ndarray): A 2D array indicating the type of trader at each position in 
          the grid (1 for Fundamentalist, 4 for Imitator).
        - price_list (list): A list of prices at each timestep.
        - fundamental_value (float): The fundamental value of the asset being traded.
        - L (int): The size of one dimension of the square grid, indicating both the width and 
          height, as the grid is L x L.

        Returns:
        - processed: A 3D array with the same shape as `transactions`, indicating the updated 
          state of each trader after processing transactions. The state is a combination of the 
          trader type and the action taken (0 for selling, 1 for holding, 2 for buying).
    """
    processed = np.zeros_like(transactions, dtype=int)
    
    for t in range(transactions.shape[0]):
        for i in range(L):
            for j in range(L):
                trader_type = 1 if trader_grid[i, j] == 1 else 4  # 1 for Fundamentalist, 4 for Imitator
                current_price = price_list[t]
                action = 1  # Default action is holding

                if trader_type == 1:  # Fundamentalist
                    if current_price < fundamental_value:  # Buy when price is below fundamental value
                        action = 2  # Buying
                    elif current_price > fundamental_value:  # Sell when price is above fundamental value
                        action = 0  # Selling

                else:  # Imitator
                    neighborhood = transactions[t, max(0, i-1):min(L, i+2), max(0, j-1):min(L, j+2)]
                    avg_action = np.mean(neighborhood)
                    if avg_action > threshold:  # Define a threshold for buying
                        action = 2  # Buying
                    elif avg_action < -threshold:  # Define a threshold for selling
                        action = 0  # Selling
                    else:
                        action = 1  # Holding


                processed[t, i, j] = trader_type + action

    return processed

In [ ]:
"""
    The CA grid represents traders as fundamentalists or imitators, with their actions (buying, 
    holding, selling) visualized in different colors. The price plot shows how the stock price changes 
    over time based on the simulation parameters. 
    The function visualizes the simulation results, including trader states and stock price evolution, 
    in an animated plot and saves the animation to a file.
"""
L = 10  
trader_grid = Functions.grid_stock_market(L, 0.5)  
fundamental_value = 100
initial_price = 100
time_steps = 250
sensitivity_constant = 0.7
constant_trading = 20
news_relevance = [0.2, 0.7]
stock_favorability = 0.01
period_length = 10
threshold = 0.0001

# Run the simulation
transactions, price_list, _, _ = Level_3_simulation(
    trader_grid, initial_price=initial_price, fundamental_value=fundamental_value, time=time_steps, L=L,
    sensitivity_constant=sensitivity_constant, constant_trading=constant_trading, news_relevance=[0.2, 0.5],
    stock_favorability=stock_favorability, period_length=period_length
)

transactions_array = np.array(transactions)
processed_transactions = process_transactions(transactions_array, trader_grid, price_list, fundamental_value, L)


# Define colors for each state
colors = [
    'darkred',  # Fundamentalist Selling
    'lightcoral',  # Fundamentalist Holding
    'red',      # Fundamentalist Buying
    'lightblue',   # Imitator Holding 
    'blue',     # Imitator Buying
    'darkblue'  # Imitator Selling 
]

cmap = ListedColormap(colors)

# Define the bounds to match the processed transactions values
bounds = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5]
norm = BoundaryNorm(bounds, cmap.N)

# Setup figure and subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Setup for CA visualization
mat = ax1.matshow(processed_transactions[0], cmap=cmap, norm=norm)
colorbar = plt.colorbar(mat, ax=ax1, boundaries=bounds[:-1], ticks=[1, 2, 3, 4, 5, 6], 
             spacing='proportional')

colorbar.ax.set_yticklabels(['F Selling', 'F Holding', 'F Buying', 'I Selling', 'I Holding', 'I Buying'], fontsize=15)

# Add text annotations
texts = [[ax1.text(j, i, 'F' if trader_grid[i, j] == 1 else 'I', ha='center', va='center', color='white', fontsize=10) 
          for j in range(L)] for i in range(L)]

# Initial setup for Price Chart
ax2.plot(price_list, color='green')  # Initial plot, will update dynamically
ax2.set_title('Price Over Time')
ax2.set_xlabel('Time Steps')
ax2.set_ylabel('Price')

def update(frame):
    # Update CA visualization
    mat.set_data(processed_transactions[frame])
    
    for i in range(L):
        for j in range(L):
            texts[i][j].set_text('F' if trader_grid[i, j] == 1 else 'I')

    # Update Price Chart
    ax2.clear()  # Clear to redraw
    ax2.plot(price_list[:frame+1], color='green')
    ax2.set_title('Price Over Time')
    ax2.set_xlabel('Time Steps')
    ax2.set_ylabel('Price')
    ax2.set_xlim(0, len(price_list))
    ax2.set_ylim(min(price_list), max(price_list))
    
    ax1.set_title(f'CA at Time Step {frame}, Price: {price_list[frame]:.2f}')

    return [mat]

fig.subplots_adjust(wspace=0.5, hspace=0.5) # Adjust the width spacing between subplots

# Create the animation
ani = FuncAnimation(fig, update, frames=len(processed_transactions), blit=False, repeat=False)
ani.save('stock_market_simulation_level3_moore_grid.mp4', writer='ffmpeg')

plt.show()

# Mean Variance of Price - Heatmaps

In [ ]:
def run_simulation_with_news_variation(L, fundamental_value, initial_price, time_steps, sensitivity_constant, constant_trading, c_fundamentalist, c_imitator, stock_favorability, period_length):
    """
        Run a stock market simulation with variations in news relevance.
        This function simulates a stock market with traders of two types: fundamentalists and imitators.
    """
    # Initialize the trader grid
    trader_grid = Functions.grid_stock_market(L, fundamentalist_probability)

    # Run the simulation with varying news relevance
    _, price_list, _, _ = Level_3_simulation(
        trader_grid, initial_price, fundamental_value, time_steps, L, 
        sensitivity_constant, constant_trading, [c_fundamentalist, c_imitator], 
        stock_favorability, period_length
    )

    return np.mean(price_list), np.std(price_list)  

### Initial Conditions for heatmaps

In [ ]:
L = 50
fundamental_value = 100
fundamentalist_probability = 0.5  # Probability of a trader being a fundamentalist
trader_grid = Functions.grid_stock_market(L, fundamentalist_probability)
initial_price = 100
time_steps = 500
constant_trading = 20
stock_favorability = 0.01
period_length = 10
num_runs = 30

## Phase transition map for Fundamentalist news vs. Imitator News

In [ ]:
# Ranges for c_fundamentalist and c_imitator
c_fundamentalist_range = np.arange(0, 1.1, 0.1)
c_imitator_range = np.arange(0, 1.1, 0.1)

# Initialize a matrix to store mean price variances
variance_matrix = np.zeros((len(c_fundamentalist_range), len(c_imitator_range)))

# Run simulations
for i, c_fundamentalist in enumerate(c_fundamentalist_range):
    for j, c_imitator in enumerate(c_imitator_range):
        variances_over_runs = []
        for _ in range(num_runs):
            _, std_price = run_simulation_with_news_variation(
                L, fundamental_value, initial_price, time_steps, sensitivity_constant, 
                constant_trading, c_fundamentalist, c_imitator, stock_favorability, period_length
            )
            variances_over_runs.append(std_price**2)  
        mean_of_variances = np.mean(variances_over_runs)
        variance_matrix[i, j] = mean_of_variances

flipped_variance_matrix = np.flipud(variance_matrix)
flipped_c_fundamentalist_range = np.flip(c_fundamentalist_range)

plt.figure(figsize=(14, 12))
sns.heatmap(flipped_variance_matrix, annot=True, fmt=".2f", cmap='plasma_r',
            xticklabels=np.round(c_imitator_range, 2),
            yticklabels=np.round(flipped_c_fundamentalist_range, 2),
            annot_kws={"size": 16})

plt.xlabel('News relevance for Imitators', fontsize=16)
plt.ylabel('News relevance for Fundamentalists', fontsize=16)
plt.title('Phase Transition Map for Mean Variance in Price', fontsize=22)
plt.tick_params(axis='x', labelsize=15)
plt.tick_params(axis='y', labelsize=15)

plt.show()

## Phase transition map for Fundamentalist news vs. Fundamentalist Probability

In [ ]:
# Ranges for c_fundamentalist and fundamentalist_probability
c_fundamentalist_range = np.arange(0, 1.1, 0.1)
fundamentalist_probability_range = np.arange(0, 1.1, 0.1)

# Initialize a matrix to store mean price variances
variance_matrix = np.zeros((len(c_fundamentalist_range), len(fundamentalist_probability_range)))
variance_matrix = variance_matrix[::-1]
# Run simulations
for i, c_fundamentalist in enumerate(c_fundamentalist_range):
    for j, fundamentalist_probability in enumerate(fundamentalist_probability_range):
        variances_over_runs = []
        for _ in range(num_runs):
            _, std_price = run_simulation_with_news_variation(
                L, fundamental_value, initial_price, time_steps, sensitivity_constant, 
                constant_trading, c_fundamentalist, 0.5, stock_favorability, period_length
            )
            variances_over_runs.append(std_price**2)
        mean_of_variances = np.mean(variances_over_runs)
        variance_matrix[i, j] = mean_of_variances
        
flipped_variance_matrix = np.flipud(variance_matrix)
flipped_c_fundamentalist_range = np.flip(c_fundamentalist_range)

# Plotting the heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(flipped_variance_matrix, annot=True, fmt=".2f", cmap='plasma_r',
            xticklabels=np.round(fundamentalist_probability_range, 2),
            yticklabels=np.round(flipped_c_fundamentalist_range, 2),
            annot_kws={"size": 16})

plt.xlabel('Fundamentalist Probability', fontsize = 16)
plt.ylabel('News relevance for Fundamentalists', fontsize = 16)
plt.title('Phase Transition Map for Mean Varaince in Price', fontsize = 22)
plt.tick_params(axis='x', labelsize=15)  
plt.tick_params(axis='y', labelsize=15) 
plt.show()


## Phase transition map for Fundamentalist news vs. Sensitivity Constant

In [ ]:
# Ranges for c_fundamentalist and sensitivity_constant
c_fundamentalist_range = np.arange(0, 1.1, 0.1)
sensitivity_constant_range = np.arange(0, 1.1, 0.1)

# Initialize a matrix to store mean price variances
variance_matrix = np.zeros((len(c_fundamentalist_range), len(sensitivity_constant_range)))
variance_matrix = variance_matrix[::-1]
# Run simulations
for i, c_fundamentalist in enumerate(c_fundamentalist_range):
    for j, sensitivity_constant in enumerate(sensitivity_constant_range):
        variances_over_runs = []
        for _ in range(num_runs):
            mean_price, price_std = run_simulation_with_news_variation(
                L, fundamental_value, initial_price, time_steps, sensitivity_constant,
                constant_trading, c_fundamentalist, 0.5, stock_favorability, period_length
            )
            variances_over_runs.append(price_std ** 2)  # Square of standard deviation to get variance
        mean_of_variances = np.mean(variances_over_runs)
        variance_matrix[i, j] = mean_of_variances

flipped_variance_matrix = np.flipud(variance_matrix)
flipped_c_fundamentalist_range = np.flip(c_fundamentalist_range)

# Plotting the heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(flipped_variance_matrix, annot=True, fmt=".2f", cmap='plasma_r',
            xticklabels=np.round(sensitivity_constant_range, 2),
            yticklabels=np.round(flipped_c_fundamentalist_range, 2),
            annot_kws={"size": 16})

plt.xlabel('Sensitivity Constant', fontsize = 16)
plt.ylabel('News relevance for Fundamentalists', fontsize = 16)
plt.title('Phase Transition Map for Mean Varaince in Price', fontsize = 22)
plt.tick_params(axis='x', labelsize=15)  
plt.tick_params(axis='y', labelsize=15) 

plt.show()